In [ ]:
!pip install tensorflowjs
!pip install TensorFlow==2.15.0
!pip install tensorflow-decision-forests==1.8.1

In [2]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Dense, InputLayer

In [3]:
tf.keras.backend.clear_session()

In [5]:
# Load dataset
df = pd.read_csv('dataset_phishing.csv')

# Map the target labels
df['status'] = df['status'].map({'phishing': 1, 'legitimate': 0})

# Filter out the non numerical columns from the dataframe
numerical_df = df.select_dtypes(include=['float64', 'int64'])

# Correlation matrix
corr_matrix = numerical_df.corr()

for i, score in enumerate(corr_matrix['status']):
  print(f"{corr_matrix.index[i]}: {score}")

length_url: 0.24858046741991785
length_hostname: 0.23832240376368571
ip: 0.3216977823874091
nb_dots: 0.20702877302594608
nb_hyphens: -0.10010749922687573
nb_at: 0.1429145682495236
nb_qm: 0.2943190708457299
nb_and: 0.17054637539522324
nb_or: nan
nb_eq: 0.23338630384578937
nb_underscore: 0.038091335717948885
nb_tilde: 0.030142327921163874
nb_percent: 0.0281012907323974
nb_slash: 0.2422699601753399
nb_star: 0.026465122927556164
nb_colon: 0.09283530688602196
nb_comma: 0.011864645370159898
nb_semicolumn: 0.10355407191497115
nb_dollar: 0.02496205711274488
nb_space: -0.004193221721663016
nb_www: -0.4434676875128603
nb_com: 0.15628353122693384
nb_dslash: 0.07260233721343713
http_in_path: 0.07077623749635599
https_token: 0.11466906528633683
ratio_digits_url: 0.3563945701626024
ratio_digits_host: 0.22433492719580636
punycode: 0.018710391971381544
port: 0.009011115578823205
tld_in_path: 0.0791465083738078
tld_in_subdomain: 0.20888415296656912
abnormal_subdomain: 0.12815984230478297
nb_subdomains:

In [6]:
# Most segnificant features that I can also extract
selected_features = ['length_url', 'length_hostname', 'ip', 'nb_dots', 'nb_qm', 'nb_eq',
                    'nb_slash', 'nb_www', 'ratio_digits_url','ratio_digits_host',
                    'tld_in_subdomain', 'prefix_suffix', 'shortest_word_host',
                    'longest_words_raw', 'longest_word_path', 'phish_hints',
                    'nb_hyperlinks', 'ratio_intHyperlinks', 'empty_title',
                    'domain_in_title']

# Select relevant features
X = df[selected_features].values
y = df['status'].values

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = Sequential()
model.add(InputLayer(input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)


Epoch 1/50
229/229 [==============================] - 2s 5ms/step - loss: 0.3727 - accuracy: 0.8447 - val_loss: 0.3162 - val_accuracy: 0.8660
Epoch 2/50
229/229 [==============================] - 1s 4ms/step - loss: 0.2865 - accuracy: 0.8800 - val_loss: 0.2964 - val_accuracy: 0.8699
Epoch 3/50
229/229 [==============================] - 1s 3ms/step - loss: 0.2620 - accuracy: 0.8942 - val_loss: 0.2764 - val_accuracy: 0.8879
Epoch 4/50
229/229 [==============================] - 1s 3ms/step - loss: 0.2488 - accuracy: 0.8995 - val_loss: 0.2656 - val_accuracy: 0.8934
Epoch 5/50
229/229 [==============================] - 1s 2ms/step - loss: 0.2338 - accuracy: 0.9058 - val_loss: 0.2570 - val_accuracy: 0.8939
Epoch 6/50
229/229 [==============================] - 1s 2ms/step - loss: 0.2229 - accuracy: 0.9092 - val_loss: 0.2526 - val_accuracy: 0.8989
Epoch 7/50
229/229 [==============================] - 1s 4ms/step - loss: 0.2163 - accuracy: 0.9118 - val_loss: 0.2429 - val_accuracy: 0.9043
Epoch 

In [7]:
import json

model.save("phishing_model.h5")

scaler_params = {
    'mean': scaler.mean_.tolist(),
    'scale': scaler.scale_.tolist()
}

with open('scaler.json', 'w') as f:
    json.dump(scaler_params, f)

/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy}")

72/72 [==============================] - 0s 3ms/step - loss: 0.2422 - accuracy: 0.9103
Test Accuracy: 0.9103237390518188
